In [1]:
import os 
import sys 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from dotenv import load_dotenv

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../.env')

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import output_chart, apply_css

In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [3]:
@cache 
def query_seasons(*args): 
    q = bs.Query.seasons(first=10000, orderBy="season", orderDirection="asc")
    df = sg.query_df([
        q.timestamp, q.harvestableIndex
    ], pagination_strategy=ShallowStrategy)
    return remove_prefix(df, "seasons_")

In [4]:
df_szns = query_seasons(1).copy()
df_szns['timestamp'] = pd.to_datetime(df_szns['timestamp'], unit='s')
df_szns['harvestableIndex'] /= 1e6
df_szns.tail()

,timestamp,harvestableIndex
7631,2022-10-10 13:00:11,5.756038e+07
7632,2022-10-10 14:00:11,5.756038e+07
7633,2022-10-10 15:00:11,5.756038e+07
7634,2022-10-10 16:00:11,5.756038e+07
7635,2022-10-10 17:00:11,5.756038e+07


In [5]:
@cache 
def query_marketplace_fills(*args):
    # Query historical farmer's market order and listing fills 
    q = bs.Query.podFills(first=10000)
    df = sg.query_df(
        [
            q.amount, 
            q.index, 
            q.start, 
            q.listing.status, 
            q.order.status, 
            q.listing.pricePerPod, 
            q.order.pricePerPod, 
            q.createdAt, 
        ],
        pagination_strategy=ShallowStrategy
    )
    df = remove_prefix(df, "podFills_")
    return df 
    

In [6]:
# history of all marketplace listing and order fills 
df_fills = query_marketplace_fills(2).copy()
df_fills = df_fills.rename(columns={"createdAt": "transaction_timestamp"})
df_fills["datetime"] = pd.to_datetime(df_fills["transaction_timestamp"], unit="s")
df_fills["date"] = pd.to_datetime(df_fills["datetime"].dt.date)
df_fills = pd.merge_asof(
    df_fills.sort_values("datetime").reset_index(drop=True), 
    df_szns.sort_values("timestamp").reset_index(drop=True), 
    left_on="datetime", 
    right_on="timestamp", 
    direction="backward", # the secret sauce right here 
)
df_fills.amount /= 10**6 
df_fills['place_in_line'] = (df_fills['index'] + df_fills.start) / 1e6 - df_fills.harvestableIndex
df_fills['price_per_pod'] = (
    df_fills.listing_pricePerPod.fillna(0) + df_fills.order_pricePerPod.fillna(0)
) / 1e6 
df_fills['type'] = df_fills.listing_status.isna().apply(lambda v: "order" if v else "listing") 
df_fills = df_fills[["date", "datetime", "amount", "place_in_line", "price_per_pod", "type"]]
df_fills.tail()

,date,datetime,amount,place_in_line,price_per_pod,type
1567,2022-10-07,2022-10-07 19:22:47,11111.111111,3.195425e+07,0.09,order
1568,2022-10-07,2022-10-07 19:23:35,28571.428571,3.196536e+07,0.07,order
1569,2022-10-07,2022-10-07 21:27:11,95.204051,7.448262e+08,0.02,listing
1570,2022-10-08,2022-10-08 14:44:59,33437.661125,3.199392e+07,0.08,order
1571,2022-10-10,2022-10-10 04:34:11,1355.743473,3.573254e+07,0.15,listing


In [7]:
# marketplace volume aggregated daily 
df_vol_daily = pd.DataFrame({
    "date": pd.date_range(df_fills["date"].min(), df_fills["date"].max()), 
})
num_dates = len(df_vol_daily)
df_vol_daily = df_vol_daily.merge(
    df_fills, how="left", on="date"
).groupby("date")["amount"].sum().reset_index()
assert len(df_vol_daily) == num_dates
df_vol_daily = df_vol_daily.rename(columns=dict(amount="pod_volume"))
df_vol_daily.pod_volume = df_vol_daily.pod_volume.fillna(0)
print(df_vol_daily.date.dtype)
df_vol_daily.tail()

datetime64[ns]


,date,pod_volume
243,2022-10-06,17250.793612
244,2022-10-07,114859.212699
245,2022-10-08,33437.661125
246,2022-10-09,0.000000
247,2022-10-10,1355.743473


In [8]:
# df_fills.date.min().dt, df_fills.date.max()

In [9]:
"""
Some notes: 
    why is vega-lite so hard ʕっ•ᴥ•ʔっ
    
    Cross filtering by temporal domain through an interval selection: 
        https://stackoverflow.com/questions/71249346/use-interval-selection-from-one-dataset-chart-to-filter-data-in-another-dataset    
    Filtering by selection when the selection's mapped encoding is binned:
        https://stackoverflow.com/questions/60994128/how-to-click-on-square-of-heatmap-to-filter-linked-bar-chart-altair
"""
width = 450
sel_point = alt.selection_single(encodings=['x'], nearest=True, on="mouseover", clear="mouseout", empty="none")
sel_brush = alt.selection_interval(
    name="brush", 
    encodings=['x'], 
    init={"x": list(pd.to_datetime([df_fills.date.min(), df_fills.date.max()]).astype(int) / 1e6)}
)

base = (
    alt.Chart(df_fills[['amount', 'place_in_line', 'price_per_pod', 'date']])
    .transform_calculate(utc_date="utcParse(datum.date, '%Y-%m-%dT%H:%M:%S')")
    .transform_filter(
        """
        data(\"brush_store\").length 
        && datum.utc_date >= data('brush_store')[0].values[0][0] 
        && datum.utc_date <= data('brush_store')[0].values[0][1]
        """
    )
) 

xbin = alt.Bin(extent=[df_fills.place_in_line.min(), df_fills.place_in_line.max()], maxbins=50)

# ------------------------ HISTOGRAM ------------------------------------

histogram_place_in_line = (
    base
    .mark_rect()
    .encode(
        x=alt.X("place_in_line:Q", bin=xbin, axis=alt.Axis(title=None, format="s")), 
        y=alt.Y("sum(amount):Q", axis=alt.Axis(title="Bin Volume (pods)", format="s")), 
        tooltip=[alt.Tooltip("sum(amount):Q", format=",d", title="Pods")],
    )
    .properties(width=width, height=100)
)

# ------------------------ HEATMAP ------------------------------------

heatmap_base = (
    base
    .properties(width=width, height=250)
)
heatmap = (
    heatmap_base
    .mark_rect()
    .encode(
        x=alt.X(
            "place_in_line:Q", 
            bin=xbin, 
            axis=alt.Axis(title="Place in Line", format="s")
        ), 
        y=alt.Y(
            "price_per_pod:Q", 
            scale=alt.Scale(domain=(0,1)), 
            bin=alt.Bin(extent=[0,1], step=.1), 
            axis=alt.Axis(title="Price Per Pod (USD)", format="$.1")
        ), 
        color=alt.Color(
            "sum(amount):Q", 
            scale=alt.Scale(type="log", scheme="viridis"), 
            legend=alt.Legend(orient="left", title="Pod Volume", direction="vertical")
        ), 
        tooltip=alt.Tooltip("sum(amount):Q", format=",",)
    )
)

heatmap_curve = (
    heatmap_base
    .encode(
        x=alt.X("place_in_line:Q", bin=xbin), 
        y=alt.Y("mean(price_per_pod)"), 
        color=alt.value("#ff3b0f"),
    )
)
yield_curve_point_size = alt.Size("sum(amount)", scale=alt.Scale(range=[8, 150]), legend=None)
# scatter plot of yield curve 
yield_curve_points = (
    heatmap_curve
    .mark_point(strokeWidth=1.25)
    .encode(
        size=yield_curve_point_size,
        tooltip=[alt.Tooltip('mean(price_per_pod):Q', title="Avg Price Per Pod", format=".2f")],
    )
    .add_selection(sel_point)
)
# single mark for currently selected point (to visually highlight selection) 
yield_curve_point_selected = (
    heatmap_curve
    .mark_point(filled=True)
    .encode(size=yield_curve_point_size, color=alt.value("#00ff00"),)
    .transform_filter(sel_point)
) 

# ------------------------ TIME AXIS ------------------------------------
time_axis = (
    alt.Chart(df_vol_daily)
    .mark_bar()
    .encode(
        x=alt.X('date:T', axis=alt.Axis(title="Date", format="%b %e, %Y", tickCount=8)), 
        y=alt.Y('pod_volume:Q', axis=alt.Axis(title="Market Volume (pods)", format="s")), 
    ).properties(width=width, height=100
    ).add_selection(sel_brush)
)

# ------------------------ TABLE ------------------------------------
cols = ['date', 'amount', 'place_in_line', 'price_per_pod', 'type']
df = df_fills[cols].copy()

df['date'] = df['date'].dt.strftime("%Y-%m-%d")
df['amount'] = df.amount.astype(int)
df['place_in_line'] = df.place_in_line.astype(int)

nrows = 27
radio_sort_col = alt.binding_radio(name="Sort Column:", options=cols)
radio_sort_dir = alt.binding_radio(name="Sort Direction:", options=['asc', 'desc'])
slider = alt.binding_range(min=1, max=len(df) - nrows, step=1, name='Scroll Offset:')

select_scroll = alt.selection_single(
    name="scroller", fields=['offset'], bind=slider, init={'offset': 1}
)
select_radio_sort_col = alt.selection_single(
    name="sortby", fields=["sort_col"], bind=radio_sort_col, init={"sort_col": "date"}
)
select_radio_sort_dir = alt.selection_single(
    name="sortdir", fields=["sort_dir"], bind=radio_sort_dir, init={"sort_dir": "asc"}
)

table_base = (
    alt.Chart(df)
    .transform_filter(
        """
        data(\"brush_store\").length 
        && toDate(datum.date) >= data(\"brush_store\")[0].values[0][0] 
        && toDate(datum.date) <= data(\"brush_store\")[0].values[0][1]
        """
    )
    .transform_joinaggregate(rc="count(*)")
    .transform_window(sort=[{"field": "date"}], frame=[None, 0], sort_date="row_number(*)")
    .transform_window(sort=[{"field": "amount"}], frame=[None, 0], sort_amount="row_number(*)")
    .transform_window(sort=[{"field": "price_per_pod"}], frame=[None, 0], sort_price_per_pod="row_number(*)")
    .transform_window(sort=[{"field": "place_in_line"}], frame=[None, 0], sort_place_in_line="row_number(*)")
    .transform_window(sort=[{"field": "type"}], frame=[None, 0], sort_type="row_number(*)")
    .transform_calculate(sort_col=f"'sort_' + {select_radio_sort_col.name}.sort_col") # this behaves strangely if not computed before calculating sort_num
    .transform_calculate(sort_num="sortdir.sort_dir[0] === 'asc' ? datum[datum.sort_col] : (datum.rc - datum[datum.sort_col] + 1)")
    .transform_fold(['date', 'amount', 'place_in_line', 'price_per_pod', 'type'])
    .transform_filter(
        f"""
        datum.sort_num >= parseInt(scroller.offset)
        && datum.sort_num < (parseInt(scroller.offset) + {nrows})
        """
    )
    .encode(
        x=alt.X(
            "key:N", 
            axis=alt.Axis(
                orient="top", 
                labelAngle=0, 
                title=None, 
                domain=False, 
                ticks=False, 
                labelFontWeight=600, 
                labelFontSize=12
            ), 
        ),
        y=alt.Y("sort_num:O", axis=None), 
    )
) 

table_rect = (
    table_base
    .mark_rect(stroke="black")
    .encode(
        color=alt.condition("datum.sort_num % 2 === 0", alt.value("#e3e3e3"), alt.value("#ffffff"))
    )
) 
table_text = (
    table_base
    .transform_calculate(
        label="""
        datum.key === 'amount' ? format(datum.value, ',') + ' pods' : 
        datum.key === 'date' ? timeFormat(toDate(datum.value), '%b %e %Y') : 
        datum.key === 'place_in_line' ? format(datum.value, ',') : 
        datum.key === 'price_per_pod' ? format(datum.value, '~') : 
        datum.key === 'type' ? (
            datum.value === 'listing' ? '🟠 ' : '🟢 ' 
        ) + datum.value :
        datum.value
        """
    )
    .mark_text()
    .encode(text='label:N')
) 

c = (
    alt.hconcat(
        alt.vconcat(
            alt.vconcat(
                histogram_place_in_line, 
                alt.layer(
                    heatmap, yield_curve_points, yield_curve_point_selected, 
                    # yield_curve_text
                ), 
                bounds="flush"
            ), 
            time_axis 
        ), 
        (
            alt.layer(table_rect, table_text)
            .add_selection(select_scroll, select_radio_sort_col, select_radio_sort_dir)
            .properties(width=500)
        )
    )
)

css_lines = [
    "div.chart-wrapper { display: flex; flex-direction: column; }", 
    """
    form.vega-bindings { 
        order: -1; 
        display: flex; 
        flex-direction: column;
        align-items: end; 
    }
    """, 
    "canvas { order: 1 }", 
    """
    div.vega-bind { 
        display: block; 
        width: 50%; 
    }
    """, 
    """
    span.vega-bind-name { 
        font-weight: 500 !important; 
        padding-right: 5px !important; 
    }
    """,
    """
    span.vega-bind-name span { 
        font-weight: 600 !important; 
        padding-right: 5px !important; 
    }
    """,
]
css = "\n".join(css_lines)
    
apply_css("")
# apply_css(css) 

c

alt.HConcatChart(...)

In [10]:
output_chart(c, css=css)

<IPython.core.display.JSON object>